In [1]:
from classify import data_pipeline, eco_selector
from resources.get_data import get_current_data
from resources.setup import get_setup
import pandas as pd

In [2]:
def validator(df):
    df = df.drop_duplicates()
    df = df[~df['text'].isna()]
    if 'vectorized' in df.columns.values:
        df = df.drop(columns=['vectorized'])
    if df.index.duplicated().any():
        df = df.reset_index()
        df = df.drop(columns = ['id'])
        df = df.rename(columns = {'index': 'id'})
    print(f"Found {len(df)} files")
    return df

# Big corpuses

## Rzepa

In [ ]:
corp = 'rzepa'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

## Wyborcza

In [ ]:
corp = 'wyborcza'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, False)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

## Gazeta Polska Codziennie

In [ ]:
corp = 'gpc'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, False)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

## Polityka

In [ ]:
corp = 'polityka'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

# Small Corpuses

## Dorzeczy

In [ ]:
corp = 'dorzeczy'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

## Wprost

In [3]:
corp = 'wprost'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_eco = pd.concat([df_eco, df_rest.drop(index=2260)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wprost
Found 445 files


## Newsweek

In [6]:
corp = 'newsweek'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_all = pd.concat([df_rest.drop(index=2051), df_eco])
df_eco = validator(df_eco)
df_all.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
newsweek


/home/ps/Code/Climate/Classification/resources/get_data.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_rest = pd.read_csv(drive_path / corpus / "results.csv", index_col = ['id', 'source'], parse_dates=['date'])


Found 378 files


## wPolityce

In [5]:
corp = 'wpolityce'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wpolityce


/home/ps/Code/Climate/Classification/resources/get_data.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_eco = pd.read_csv(drive_path / corpus / "eco_result.csv", index_col=['id', 'source'], parse_dates=['date'])
/home/ps/Code/Climate/Classification/resources/get_data.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_non_eco = pd.read_csv(drive_path / corpus / "non_eco_result.csv", index_col = ['id', 'source'], parse_dates=['date'])


Found 398 files
